# Sonderübung 2

## Imports

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import scipy
from gurobipy import Model, GRB, Var

### Achtung: 
die untenstehenden Befehle funktionieren nur mit einer aktuellen pandas Version: falls beim pandas.drop() Befehl, ein Fehler auftritt , dann schauen, dass pandas geupdated wird zb über "pip install pandas --upgrade" (Version 1.4.2. funktioniert), danach das Notebook komplett neustarten und nicht nur die Importzelle neu ausführen

## Parameter für das Optimierungsproblem

In [2]:
shares = ['AAPL', 'MSFT', 'XOM', 'JNJ', 'GE', 'BRK-B', 'FB', 
          'AMZN', 'WFC','T','GOOGL','PG','GOOG','VZ','PFE',
          'CVX','KO','HD','DIS','BAC','MRK','V','PM','CMCSA',
          'INTC','PEP','CSCO','C','GILD','IBM']

In [3]:
historical_prices = pd.read_excel('historical_prices.xlsx', index_col=0)
historical_returns = historical_prices / historical_prices.shift(1) - 1
historical_returns = historical_returns.drop(['2015-05-13'])
mu = historical_returns.mean()*252 # Estimation of mu
sigma = historical_returns.cov()*252 # Estimation of covariance-matrix

In [4]:
sigma.iloc[0:8,0:8]

,AAPL,MSFT,XOM,JNJ,GE,BRK-B,FB,AMZN
AAPL,0.077915,0.045575,0.031541,0.020971,0.028955,0.027391,0.040927,0.040994
MSFT,0.045575,0.072776,0.027717,0.025104,0.032222,0.029367,0.048484,0.051320
XOM,0.031541,0.027717,0.059183,0.022321,0.028922,0.027715,0.024145,0.024970
JNJ,0.020971,0.025104,0.022321,0.026636,0.019970,0.021105,0.022804,0.023365
GE,0.028955,0.032222,0.028922,0.019970,0.043824,0.026067,0.029822,0.028163
BRK-B,0.027391,0.029367,0.027715,0.021105,0.026067,0.031443,0.026365,0.025164
FB,0.040927,0.048484,0.024145,0.022804,0.029822,0.026365,0.106992,0.075159
AMZN,0.040994,0.051320,0.024970,0.023365,0.028163,0.025164,0.075159,0.121713


In [5]:
Sigma = sigma.values
Mu = mu.values
lamb = 10
n = Mu.size

## Aufgabe 2 b) 
Zeigen Sie, dass die Zielfunktion von $P$ mit den gegebenen Daten $\Sigma, \mu$ und $\lambda$ gleichmäßig konvex ist.

In [6]:
eigenvalues, _ = np.linalg.eig(lamb*Sigma)
print(min(eigenvalues))

0.007788373017431655


## 2 c) Frank Wolfe
Implementieren Sie das Verfahren aus dem Algorithmus auf dem Aufgabenblatt. Geben Sie am Ende aus, wie viel in welche Aktien investiert wird. Beachten Sie die Hinweise auf dem Aufgabenblatt.

In [7]:
eps = 1e-4

In [8]:
def problem_S(x_k,d_k):
    m = Model("S")
    m.Params.LogToConsole = 0
    
    t = m.addVar(vtype = GRB.CONTINUOUS, lb=0, ub=1, name = "t")
    
    lin = m.addMVar(n, vtype = GRB.CONTINUOUS, lb=float('-inf'), ub=float('inf'), name = "t")
    for i in range(n):
        m.addConstr(x_k[i]+t*d_k[i] == lin[i])
        
    m.setObjective(-Mu @ lin + 1/2 * lamb * (lin @ Sigma @ lin))
    
    m.optimize()
    return m.x[0], m.ObjVal

In [9]:
def problem_Q(xk):
    m = Model("Q")
    m.Params.LogToConsole = 0
    
    x = m.addMVar(len(shares), lb=0, name="x")
    m.addConstr(x.sum() == 1)
    
    vec = -Mu+lamb*Sigma@xk
    m.setObjective(sum((x[i]-xk[i])*vec[i] for i in range(len(vec))))
    
    m.optimize()
    return m.x, m.ObjVal

In [10]:
k = 0
x_k = np.concatenate([np.array([1]),np.repeat(0,29)])
y_k, v_k = problem_Q(x_k)

while v_k < -eps:
    d_k = y_k-x_k
    t_k, _ = problem_S(x_k=x_k, d_k=d_k)
    x_k = x_k + t_k*d_k
    k += 1
    y_k, v_k = problem_Q(x_k)

x_quer = x_k
Obj_Frank_Wolfe = -Mu@x_quer+1/2*lamb*(x_quer@Sigma@x_quer)
print(f"Lösung nach {k} iterationen: x={x_quer}, Objective Value={Obj_Frank_Wolfe}")

Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-28
Lösung nach 56 iterationen: x=[9.79351738e-10 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.61782108e-01 3.02403509e-01
 0.00000000e+00 2.40648825e-01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.95165557e-01 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00], Objective Value=-0.16481089931376122


In [18]:
for k, v in sorted(dict(zip(shares,x_quer)).items(), key=lambda x: x[1], reverse=True): print(f"{k}: {v}")

AMZN: 0.3024035091391453
PM: 0.29516555736119554
T: 0.24064882462566003
FB: 0.16178210789464734
AAPL: 9.793517381744017e-10
MSFT: 0.0
XOM: 0.0
JNJ: 0.0
GE: 0.0
BRK-B: 0.0
WFC: 0.0
GOOGL: 0.0
PG: 0.0
GOOG: 0.0
VZ: 0.0
PFE: 0.0
CVX: 0.0
KO: 0.0
HD: 0.0
DIS: 0.0
BAC: 0.0
MRK: 0.0
V: 0.0
CMCSA: 0.0
INTC: 0.0
PEP: 0.0
CSCO: 0.0
C: 0.0
GILD: 0.0
IBM: 0.0


## 2 d) Python Lösung
Lösen Sie $P$ direkt mit Python und vergleichen Sie ihren Optimalpunkt mit dem aus Teil c). Ist dieses Ergebnis zu erwarten? Verwenden Sie Aufgabenteil b).

In [12]:
m = Model("d)")
x = m.addMVar(len(shares), lb=0)
m.addConstr(x.sum() == 1)
m.setObjective(-Mu@x+1/2*lamb*(x@Sigma@x))
m.optimize()
Obj_Gurobi = m.ObjVal
x_gurobi = m.x

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 1 rows, 30 columns and 30 nonzeros
Model fingerprint: 0x311a088c
Model has 465 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-03, 6e-01]
  QObjective range [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 1 rows, 30 columns, 30 nonzeros
Presolved model has 465 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 29
 AA' NZ     : 4.350e+02
 Factor NZ  : 4.650e+02
 Factor Ops : 9.455e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.11394286e+05 -4.15036547e+05  2.30e+04 5.81e-01  9.99e+05     0s
   1   9.59011366e+04 -9.65538749e+04  1.63e+03 5.4

### Vergleich

In [19]:
print(x_gurobi - x_quer)

[-9.78928197e-10  1.21501025e-12  1.85794084e-12  7.13367018e-11
  3.81297657e-12  1.29544439e-12  1.98562512e-04 -9.81524721e-05
  7.07885654e-13 -1.30282457e-03  7.63909988e-09  2.21215687e-12
  8.66401490e-09  2.17110213e-12  1.61567733e-12  9.06603247e-13
  1.33644486e-11  1.84084002e-10  9.43350960e-13  6.03210826e-13
  9.34471375e-13  3.22469235e-12  1.20239891e-03  3.18715080e-12
  8.62935022e-13  4.01115417e-12  7.73246369e-13  5.18884801e-13
  5.76986042e-13  7.56002811e-13]


In [21]:
print(f"Objective Value:\nFrank Wolfe Verfahen: {Obj_Frank_Wolfe}\nGurobi: {Obj_Gurobi}\nDifference:  {Obj_Frank_Wolfe-Obj_Gurobi}")

Objective Value:
Frank Wolfe Verfahen: -0.16481089931376122
Gurobi: -0.1648110676775003
Difference:  1.6836373908923896e-07
